In [33]:
import numpy as np
from scipy.integrate import quad
from prettytable import PrettyTable
import matplotlib.pyplot as plt

In [34]:
def f(x):
    return 3.7 * np.cos(1.5 * x) * np.exp(-4 * x / 3) + 2.4 * np.sin(4.5 * x) * np.exp(2 * x / 3) + 4

def f_p(x):
    return (3.7 * np.cos(1.5 * x) * np.exp(-4 * x / 3) + 2.4 * np.sin(4.5 * x) * np.exp(2 * x / 3) + 4) * ((2.3 - x) ** (-3 / 5))

узлы + моменты

In [35]:
def nodes(z_1, z_2):
    x = [z_1, (z_1 + z_2) / 2, z_2]
    return x

def moment(z_1, z_2, j):
    return ((2.3 - z_1) ** (j - (3 / 5) + 1) - (2.3 - z_2) ** (j-(3 / 5) + 1)) / (j - (3 / 5) + 1)

In [36]:
def H_NORM(a, b):
    h = np.array([(b - a) / 2])
    h = np.append(h, h[0] / 2)
    h = np.append(h, h[1] / 2)
    return h

def H_OPT(a, b):
    h = np.array([(b - a) / 2])
    h = np.append(h, h[0] / 2)
    h = np.append(h, h[1] / 2)
    r = 2
    S = np.array([S_found_1(h[0], a, b), S_found_1(h[1], a, b), S_found_1(h[2], a, b)])
    R = R_found(r, S, h)
    m = -np.log(abs((S[len(h) - 1] - S[len(h) - 2]) / (S[len(h) - 2] - S[len(h) - 3]))) / np.log(2)
    h_opt = np.array([h[0]*(1e-6/abs(R[0]))**(1/m)])
    h_opt = np.append(h_opt, h_opt[0] / 2)
    h_opt = np.append(h_opt, h_opt[1] / 2)
    return h_opt

Ньютон-Котс

In [37]:
def Nuthon_Kots(a_1, b_1, n):
    mu = np.array([moment(a_1, b_1, i) for i in range(n)])
    z_1 = 2.3 - b_1
    z_2 = 2.3 - a_1
    t = nodes(z_1, z_2)
    P = np.array([np.array([t[i] ** s for i in range(n)]) for s in range(n)])
    A = np.linalg.solve(P, mu)
    result = 0
    for i in range(n):
        result += A[i] * f(2.3 - t[i])
    return result

In [38]:
def S_found_1(h, a, b):
    j = int((b-a)/h)
    s = 0
    for i in range(j):
        a_1 = a + h * i
        b_1 = a + h * (i + 1)
        s += Nuthon_Kots(a_1, b_1, 3)
    return s

def R_found(r, S, h):
    m = -np.log(abs((S[len(h)-1]-S[len(h)-2])/(S[len(h)-2]-S[len(h)-3])))/np.log(2)
    print(m)
    P = np.array([np.array([h[j]**(m+i)-h[j+1]**(m+i) for i in range (r)]) for j in range (len(h)-1)])
    y = np.array([S[i+1]-S[i] for i in range (len(h)-1)])

    C = np.linalg.solve(P, y)
    R = []

    for j in range (len(h)):
        summ = 0
        for i in range(r):
            summ += C[i] * h[j] ** (m + i)
        R.append(summ)
    return R

def Nuthon_Kots_error(a, b, h):
    r = 2
    S = np.array([S_found_1(h[0], a, b), S_found_1(h[1], a, b), S_found_1(h[2], a, b)])
    R = R_found(r, S, h)
    while abs(R[len(R) - 1]) > 1e-6:
        r += 1
        h = np.append(h, h[r - 1] / 2)
        S = np.append(S, S_found_1(h[r], a, b))
        R = R_found(r, S, h)
    return(h[len(h)-1])


In [39]:
a = 1.8
b = 2.3
h = H_NORM(a, b)
h_n_k = Nuthon_Kots_error(a, b, h)
print()
h_opt = H_OPT(a, b)
h_n_k_opt = Nuthon_Kots_error(a, b, h_opt)

znach1 = quad(f_p, a, b)[0]
znach2 = S_found_1(h_n_k, a, b)
print(abs(znach2-znach1), znach1, znach2)
table = PrettyTable(["", "Длина шага ℎ разбиения К.Ф. Ньютона-Котса"])
table.add_row(["Без вычисления h_opt", h_n_k])
table.add_row(["С вычислением h_opt", h_n_k_opt])
print(table)

3.3023182088857097
3.3586418075592754
3.378505534440835
3.387143191102873

3.3023182088857097
3.8235821775811867
3.5985641644309396e-07 1.1851419749802814 1.1851423348366978
+----------------------+-------------------------------------------+
|                      | Длина шага ℎ разбиения К.Ф. Ньютона-Котса |
+----------------------+-------------------------------------------+
| Без вычисления h_opt |           0.0078124999999999965           |
| С вычислением h_opt  |             0.0024902908991091            |
+----------------------+-------------------------------------------+
